# Identify vehicle traffic density with yolov8

## Setup

In [ ]:
%pip install -r requirements.txt

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

## Training

In [ ]:
model = YOLO("./models/yolov8m.yaml")

In [ ]:
data_path="./datasets/dataset.yaml"
project = "./logs"

In [ ]:
results = model.train(data=data_path, epochs=200, imgsz=96, project=project, pretrained=False, batch=4)

## Resuming interrupted trainings

In [ ]:
model_path='./models/last.pt'

In [ ]:
model = YOLO(model_path)  # load a partially trained model

results = model.train(resume=True)

## Make a prediction

In [ ]:
model = YOLO("models/yolov8x-seg.pt")

### Road segmentation

In [ ]:
def road_segmnent(image):
  # Convert imae to hsv
  hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

  # Segment green object
  lower_blue = np.array([40, 50, 50])
  upper_blue = np.array([90, 255, 255])
  mask = cv2.inRange(hsv, lower_blue, upper_blue)

  # Cut the ROI
  active_pixels = np.stack(np.where(mask))
  top_left = np.min(active_pixels, axis=1).astype(np.int32)
  bottom_right = np.max(active_pixels, axis=1).astype(np.int32)
  y1, x1 = top_left
  y2, x2 = bottom_right
  mask = mask[y1:, x1:x2]

  # Extract the largest component
  mask = mask.astype('uint8')
  mask = cv2.bitwise_not(mask)
  mask = mask.astype('uint8')
  nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
  sizes = stats[:, -1]
  max_label = 1
  max_size = sizes[1]
  for i in range(2, nb_components):
    if sizes[i] > max_size:
      max_label = i
      max_size = sizes[i]
  mask = np.zeros(output.shape)
  mask[output == max_label] = 255

  # Connect
  mask = mask.astype('uint8')
  mask = cv2.bitwise_not(mask)
  mask = mask.astype('uint8')
  cnts = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  for c in cnts:
    area = cv2.contourArea(c)
    if area < 10:
      cv2.drawContours(mask, [c], -1, (0, 0, 0), -1)
  # Morph close and invert image
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 30))
  mask = 255 - cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

  # Extract the largest component
  mask = mask.astype('uint8')
  nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
  sizes = stats[:, -1]
  max_label = 1
  max_size = sizes[1]
  for i in range(2, nb_components):
    if sizes[i] > max_size:
      max_label = i
      max_size = sizes[i]
  mask = np.zeros(output.shape)
  mask[output == max_label] = 255

  # Cut the ROI
  mask = mask.astype('uint8')
  mask = cv2.bitwise_not(mask)
  active_pixels = np.stack(np.where(mask))
  top_left = np.min(active_pixels, axis=1).astype(np.int32)
  bottom_right = np.max(active_pixels, axis=1).astype(np.int32)
  y3, x3 = top_left
  y4, x4 = bottom_right
  mask = mask[y3:, x3:x4]

  return mask, y1, x1, y2, x2, y3, x3, y4, x4

### Segmentation with images

In [ ]:
image = (r'predict_data/14.png')

cls_names = model.names
cap = cv2.imread(image)

# count road pixel
road_mask, y1, x1, y2, x2, y3, x3, y4, x4 = road_segmnent(cap)

cap = cap[y1:,x1:x2]
cap = cap[y3:,x3:x4]

results = model.predict(cap, device="mps", classes=[0, 1, 2, 3, 5, 7])
result = results[0]

boxes = np.array(result.boxes.xyxy.cpu(), dtype="int")
classes = np.array(result.boxes.cls.cpu(), dtype="int")
masks = result.masks.xy

for cls, box, mask in zip(classes, boxes, masks):
  (x, y, x2, y2) = box
  points = np.int32([mask])
  cv2.fillPoly(cap, points, (0, 0, 0))

# count vehicle pixel
gray_image = cv2.cvtColor(cap, cv2.COLOR_BGR2GRAY)
ret, thresh_binary = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY)

p = np.where(road_mask == 0)
p1 = np.where(thresh_binary == 0)
# cap[p] = 0
# cap[p1] = 255

# Calculate pixel
road_pixel = np.sum(road_mask == 0)*0.5
vehicle_pixel = np.sum(thresh_binary == 0)
P = vehicle_pixel/road_pixel*100

print(f'Road: {road_pixel}')
print(f'Vehicle: {vehicle_pixel}')
print(f'P = {P}%')

if P>50:
  print('High')
elif (P>=20) & (P<50):
  print('Medium')
else:
  print('Low')

cv2.imshow("Img", cap)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### Segmentation with videos

In [ ]:
cap = cv2.VideoCapture("./predict_data/pexels-christopher-schultz-5927708 (720p).mp4")

cls_names = model.names

while True:
  ret, frame = cap.read()
  if not ret:
    break

  road_mask, y1, x1, y2, x2, y3, x3, y4, x4 = road_segmnent(frame)
  road_pixel = np.sum(road_mask == 0)

  frame = frame[y1:,x1:x2]
  frame = frame[y3:,x3:x4]

  results = model.predict(frame, device="mps", classes=[1, 2, 3, 5, 7])
  result = results[0]

  boxes = np.array(result.boxes.xyxy.cpu(), dtype="int")
  classes = np.array(result.boxes.cls.cpu(), dtype="int")
  masks = result.masks.xy

  for cls, box, mask in zip(classes, boxes, masks):
    (x, y, x2, y2) = box
    points = np.int32([mask])
    cv2.fillPoly(frame, points, (0, 0, 0))

  # count vehicle pixel
  gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  ret, thresh_binary = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY)
  vehicle_pixel = np.sum(thresh_binary == 0)

  p = np.where(road_mask == 0)
  p1 = np.where(thresh_binary == 0)
  frame[p] = 0
  frame[p1] = 255  

  print(f'Road: {road_pixel}')
  print(f'Vehicle: {vehicle_pixel}')
  print(f'P = {vehicle_pixel/road_pixel*100}%')

  cv2.imshow("Video", frame)

  key = cv2.waitKey(1)
  if key == 27:  # Press Esc to close
    break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)